## Web scraping
##### Import libaries

In [16]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

#### Get the source of the data

In [117]:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#### Parse using lxml

In [10]:
soup = BeautifulSoup(source,'lxml')

#### Get the table element

In [13]:
 table = soup.find_all('table')[0] 

#### The postcode is repeated by increment by 3, starting at position 0
#### Append the postcode to the list PCodeList

In [128]:
PCodeList=[]
for i, tag in enumerate(soup.table.find_all('td')):
    if i in range(0,len(soup.table.find_all('td')),3):
        PCodeList.append(tag.text.strip())

#### The Borough is repeated by increment by 3, starting at position 1
#### Append the Borough to the list BoroughList

In [129]:
BoroughList=[]
for i, tag in enumerate(soup.table.find_all('td')):
    if i in range(1,len(soup.table.find_all('td')),3):
        BoroughList.append(tag.text.strip())

#### The Neighborhood is repeated by increment by 3, starting at position 2
#### Append the Neighboorhood to the list NBhoodList

In [130]:
NBhoodList=[]
for i, tag in enumerate(soup.table.find_all('td')):
    if i in range(2,len(soup.table.find_all('td')),3):
        NBhoodList.append(tag.text.strip())

#### Create the datadrame using the three lists that we created

In [131]:
data = [('PostalCode', PCodeList),
         ('Borough', BoroughList),
         ('Neighborhood', NBhoodList)]
df = pd.DataFrame({'PostalCode':PCodeList,'Borough':BoroughList, 'Neighborhood':NBhoodList})
# check put first 5 rows
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Drops the rows for which the Borough is 'Not assigned'

In [120]:
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)

#### Checking out the rows that have value 'Not assigned' in the column Neighborhood

In [121]:
# just checking the rows that have 'Not assigned' Neighborhood
df[df['Neighborhood']=='Not assigned']

,PostalCode,Borough,Neighborhood
8,M7A,Queen's Park,Not assigned


#### Assign the value of Borough to Neighboord if the value of the latter is 'Not assigned' 

In [122]:
 df.loc[df['Neighborhood']=='Not assigned','Neighborhood'] = df.loc[df['Neighborhood']=='Not assigned','Borough']

#### Combine the rows with the same postal code

In [123]:
df1 = df.groupby(['PostalCode','Borough'], sort = False).agg(lambda x: ','.join(x))

#### Check out first five rows of the final data

In [132]:
df1.head()

,,Neighborhood
PostalCode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront,Regent Park"
M6A,North York,"Lawrence Heights,Lawrence Manor"
M7A,Queen's Park,Queen's Park


#### print the number of rows of the dataframe

In [127]:
print(df1.shape[0])

103
